In [ ]:
# colab 실행시 필요한 다운로드

#Fast api 
#!pip install fastapi nest-asyncio pyngrok uvicorn
# !pip install aiofiles

#qr 코드
#!pip install qrcode

# yolov4 사용 하려면 4.4.0 버전 이상 OpenCv를 사용해야 함
#!pip install --upgrade opencv-python

In [1]:
# 필요한 라이브러리 당겨오기 
from typing import Optional
# Pydantic을 이용해 파이썬 표준 타입으로 분문을 선언 
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# json, qrcode 라이브러리 가져오기 
import json
import qrcode

# FastAPI  및 웹 관련 라이브러리 가져오기 
from fastapi import FastAPI
from fastapi import Request
from fastapi.staticfiles import StaticFiles #new
from fastapi.templating import Jinja2Templates
from fastapi import HTTPException
from core.config import settings
from apis.general_pages.route_homepage import general_pages_router
from fastapi.responses import HTMLResponse

# 모델 2 필요한 것 
import cv2
import numpy as np
# from google.colab.patches import cv2_imshow # colab에서는 이걸 사용해야함
from glob import glob

# json 데이터 가져오기 
with open('account.json', 'r') as f:
    account = json.load(f)
with open('pet_num.json', 'r') as fp:
    data = json.load(fp)


classes = []
# class의 이름들
with open("./data/custom.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# py 파일 가져오기 


# 경고 무시하기 
import warnings
warnings.filterwarnings('ignore')

In [2]:
# ocr로 사용자의 정보를 입력 받는다 
# Module 1 : 사진파일 >> 텍스트 
def call_picture(pic):
  import json
  import base64
  import requests

  f = open(pic, "rb")
  img = base64.b64encode(f.read())

  # 본인의 APIGW Invoke URL로 치환
  URL = "https://753be9f191e84997b6a1cb14d20ae459.apigw.ntruss.com/custom/v1/12797/d9761cebea7a1a7d2cae3aa32323831aaa6470b73fe9316b6b54ad0fead46926/infer"
      
  # 본인의 Secret Key로 치환
  KEY = "R3JJZWN3cW1EaXFLcXRocGV5ek5ORERLQlJ2VEFQWFA="
      
  headers = {
      "Content-Type": "application/json",
      "X-OCR-SECRET": KEY
  }
      
  data = {
      "version": "V2",
      "requestId": "string",
      "timestamp": 0,         # 현재 시간값
      "lang":"ko",
      "images": [
          {
              "name": "sample_image",
              "format": "png",
              "data": img.decode('utf-8')
            # "templateIds": [400]  # 설정하지 않을 경우, 자동으로 템플릿을 찾음 
          }
      ]
  }
  data = json.dumps(data)
  response = requests.post(URL, data=data, headers=headers)
  res = json.loads(response.text)
  #name = res['images'][0]['matchedTemplate']['name']     # 약이름만 추출 
  id = res['images'][0]['title']['inferText']
  return id

In [15]:
# 카드에서 사용자 ID 인식하기 (OCR 이용)
call_picture('pet_card2.jpg')

'PET102'

해야하는 일 : 성능이 조금더 높은 h5 모델 불러와서 사용해보기 

In [5]:
# Module 2 : .h5 모델 불러오기 
from tensorflow.python.keras.models import load_model
model = load_model('PET_model_V3.h5')

2021-12-02 19:09:29.576411: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Module 2 : pass or fail 알려주고 확인하기 
# 올바르게 분류 됐으면 "올바르게 분류 됐습니다."
# 올바르게 분류 되지 않았으면 "방법에 맞게 분류해주세요."

# 작업 해야함 

In [5]:
# 아이디, 페트병 개수, 리워드 현황 출력 
# json 파일로 변경사항 저장 
def pet_sum(user_id:str, pet_num:int):
  if user_id in list(data.keys()):
    data[user_id]+= pet_num
    with open('pet_num.json', 'w') as fp:
      json.dump(data,fp)
    return data[user_id]
  else:
    new_m_p = {user_id : pet_num}
    data.update(new_m_p)
    with open('pet_num.json', 'w') as fp:
      json.dump(data,fp)
    return data[user_id]

In [6]:
# Module 3 : API로 결과 출력 

templates = Jinja2Templates(directory="templates")

def include_router(app):
    app.include_router(general_pages_router)


def configure_static(app):  #new
    app.mount("/static", StaticFiles(directory="static"), name="static")


def start_application():
    app = FastAPI(title=settings.PROJECT_NAME,version=settings.PROJECT_VERSION)
    include_router(app)
    configure_static(app) #new
    return app

app = start_application()

# 웹페이지만 나옴 
@app.get('/login/')
def login(request: Request):
    return templates.TemplateResponse("auth/login.html", {"request": request})

# 페트병 개수 
@app.get('/pet/', response_class=HTMLResponse)
async def pet(request: Request):
    # 사용자 정보 
    card = 'pet_card.jpg'
    user_id = call_picture(card)
    # 페드병 개수 
    image_list = glob('./image/*/*/*.jpg')
    pet_num = pet_total(image_list)

    # 결과
    result = pet_sum(user_id, pet_num)
    img = qrcode.make(f'{ngrok_tunnel.public_url}/pet/?pet_num={pet_num}')
    img.save('result.png')
    return templates.TemplateResponse("main.html", {"request": request, 'id':user_id, 'pet_num':result})


ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [13712]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://c8fb-114-203-149-236.ngrok.io


[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU
[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU
[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU
[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU


[clean(0)] conf: 0.4139142632484436 / x: 774 / y: 403 / width: 1175 / height: 3670


[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU


...

[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU
[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU
[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU
[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU


INFO:     127.0.0.1:62779 - "GET /pet/ HTTP/1.1" 200 OK


[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU


INFO:     127.0.0.1:62779 - "GET /static/images/nolabel.png HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [13712]
